In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


In [4]:
# Define paths 
train_dir = r'C:\Users\mango\Downloads\VS Code Projects\Brain Tumor\Brain Tumor data set\Train'
validatio_dir = r'C:\Users\mango\Downloads\VS Code Projects\Brain Tumor\Brain Tumor data set\Validation'

# Set image size and batch size
image_size = (128, 128)
batch_size = 32

# Image data generator for training set with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Splitting validation data
)

# Image data generator for validation set
validation_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

# Loading training data
train_generator = train_datagen.flow_from_directory(
   train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

# Loading validation data
validation_generator = validation_datagen.flow_from_directory(
   validatio_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 203 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [5]:
# Initialize the CNN
model = Sequential()

# Convolutional Layer 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 128)      0

In [6]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=25  # You can adjust the number of epochs
)


Epoch 1/25
6/6 [==============================] - 7s 931ms/step - loss: 0.7499 - accuracy: 0.5439 - val_loss: 0.6553 - val_accuracy: 0.6562
Epoch 2/25
6/6 [==============================] - 4s 626ms/step - loss: 0.6678 - accuracy: 0.6140 - val_loss: 0.6408 - val_accuracy: 0.5625
Epoch 3/25
6/6 [==============================] - 4s 664ms/step - loss: 0.6656 - accuracy: 0.6199 - val_loss: 0.6954 - val_accuracy: 0.5625
Epoch 4/25
6/6 [==============================] - 5s 800ms/step - loss: 0.6584 - accuracy: 0.6250 - val_loss: 0.5935 - val_accuracy: 0.6562
Epoch 5/25
6/6 [==============================] - 4s 811ms/step - loss: 0.6637 - accuracy: 0.6374 - val_loss: 0.6199 - val_accuracy: 0.5938
Epoch 6/25
6/6 [==============================] - 4s 699ms/step - loss: 0.6506 - accuracy: 0.6199 - val_loss: 0.5921 - val_accuracy: 0.6250
Epoch 7/25
6/6 [==============================] - 5s 750ms/step - loss: 0.6399 - accuracy: 0.6316 - val_loss: 0.4929 - val_accuracy: 0.7500
Epoch 8/25
6/6 [====

In [7]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


2/2 [==============================] - 0s 72ms/step - loss: 0.4864 - accuracy: 0.7600
Validation Loss: 0.4864286780357361
Validation Accuracy: 0.7599999904632568


In [8]:
# Save the model
model.save('brain_tumor_cnn_model.h5')


In [19]:
# Load the model
loaded_model = tf.keras.models.load_model('brain_tumor_cnn_model.h5')

# Make predictions
test_image_path = r"C:\Users\mango\Downloads\VS Code Projects\Brain Tumor\Brain Tumor data set\Validation\yes\Y31.jpg"
img = tf.keras.preprocessing.image.load_img(test_image_path, target_size=image_size)
img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_array = tf.expand_dims(img_array, axis=0)

predictions = loaded_model.predict(img_array)
print(f'Prediction (0: No Tumor, 1: Tumor): {predictions[0][0]}')


1/1 [==============================] - 0s 82ms/step
Prediction (0: No Tumor, 1: Tumor): 0.6697573065757751


In [18]:
# Load the model
loaded_model = tf.keras.models.load_model('brain_tumor_cnn_model.h5')

# Evaluate the model on the validation dataset
eval_result = loaded_model.evaluate(validation_generator)
print(f'Validation Loss: {eval_result[0]}')
print(f'Validation Accuracy: {eval_result[1]}')

# Predict on the validation dataset
predictions = loaded_model.predict(validation_generator)
predicted_classes = (predictions > 0.5).astype(int).flatten()  # Convert probabilities to binary classes

# Get true classes
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Compare predictions with true classes
from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))

2/2 [==============================] - 0s 71ms/step - loss: 0.4864 - accuracy: 0.7600
Validation Loss: 0.4864286780357361
Validation Accuracy: 0.7599999904632568
2/2 [==============================] - 0s 75ms/step
Classification Report:
              precision    recall  f1-score   support

          no       0.47      0.42      0.44        19
         yes       0.67      0.71      0.69        31

    accuracy                           0.60        50
   macro avg       0.57      0.57      0.57        50
weighted avg       0.59      0.60      0.60        50

Confusion Matrix:
[[ 8 11]
 [ 9 22]]
